# Регрессия

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

С 1ого октября отменяется НДС на бриллианты, теперь они становятся новым инвестиционным инструментом. Давайте сделаем модель ценообразования для них.

Скачайте датасет diamonds.csv

В нем представлены характеристики бриллиантов и их цены с сайта jamesallen (B2C площадка) с 2022-07-01

**Описание полей**


* fluor - флуорисценуия (свойство камня светиться в лучах ультр)
* symmetry - показатель симметричности
* platform - название платформы, где был размещен камень
* quality_group - составной показатель из cut polish symmetry
* size_group - каратно весовая группа
* big_size_group - каратно-весовая группа
* shape - форма
* color - цвет
* clarity - чистота
* cut - качество огранки (может быть только у круглых камней)
* polish - полировка
* id - номер камня
* date - дата
* price - цена
* carat - кол-во карат
* tablepercent - размер площадки по отношению ширине
* price_per_carat - цена за карат
* z - длина (diameter)
* x - ширина
* depth_perc - отношение высоты к ширине
* y - высота



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = 'drive/MyDrive/'

Mounted at /content/drive


In [ ]:
df = pd.read_csv(path + 'diamonds.csv', index_col=0)
# df = pd.read_csv('diamonds.csv', index_col=0)
df

,fluor,symmetry,platform,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y
135269,NONE,EX,jamesallen,PS,F,SI1,NaN,EX,13870838,202206,12850.0,1.55,8290.32,10.29,6.41,62.0,3.97420
48477,MED,EX,jamesallen,RD,H,VVS2,EX,EX,11725253,202207,7510.0,1.02,7362.75,6.41,6.45,62.5,4.03125
236786,NONE,EX,jamesallen,EM,H,IF,NaN,EX,14444347,202205,21220.0,2.01,10557.21,8.60,6.37,65.0,4.14050
235781,NONE,EX,jamesallen,RD,E,VS2,EX,EX,14438434,202207,8660.0,1.00,8660.00,6.39,6.44,61.4,3.95416
277744,NONE,VG,jamesallen,RD,F,VS1,VG,EX,14615276,202206,8480.0,1.00,8480.00,6.28,6.36,62.3,3.96228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20595,NONE,EX,jamesallen,RD,D,VS2,EX,EX,7535786,202207,8700.0,0.97,8969.07,6.36,6.32,62.2,3.93104
282754,NONE,EX,jamesallen,RD,D,VS2,EX,EX,14631732,202206,6650.0,0.90,7388.89,6.13,6.18,61.7,3.81306
141550,NONE,EX,jamesallen,RD,J,SI2,EX,EX,13932645,202205,11570.0,1.52,7611.84,7.36,7.39,61.0,4.50790
71237,STG,EX,jamesallen,RD,G,VS1,EX,EX,12699310,202207,8220.0,1.09,7541.28,6.55,6.60,62.0,4.09200


Хоти предсказать price_per_carat

## Задание 1: Очистка

Не все камни успевают продаться за один месяц, поэтому в таблице есть повторы. Объедините данные по одному камню: подумайте, как лучше это сделать, какую цену брать.

Попробуйте найти аномалии: вдруг цена на некоторые камни сильно меняется (то есть продавец сам не знает, по какой цене их продавать). Также убедитесь, что остальные параметры камня не меняются.

In [ ]:
# Посмотрим на столбец platform
df.platform.unique()


array(['jamesallen', nan], dtype=object)

In [ ]:
# Удалим столбец platform - тк там одно значение для всего набора
del df['platform']
df

,fluor,symmetry,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y
135269,NONE,EX,PS,F,SI1,NaN,EX,13870838,202206,12850.0,1.55,8290.32,10.29,6.41,62.0,3.97420
48477,MED,EX,RD,H,VVS2,EX,EX,11725253,202207,7510.0,1.02,7362.75,6.41,6.45,62.5,4.03125
236786,NONE,EX,EM,H,IF,NaN,EX,14444347,202205,21220.0,2.01,10557.21,8.60,6.37,65.0,4.14050
235781,NONE,EX,RD,E,VS2,EX,EX,14438434,202207,8660.0,1.00,8660.00,6.39,6.44,61.4,3.95416
277744,NONE,VG,RD,F,VS1,VG,EX,14615276,202206,8480.0,1.00,8480.00,6.28,6.36,62.3,3.96228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20595,NONE,EX,RD,D,VS2,EX,EX,7535786,202207,8700.0,0.97,8969.07,6.36,6.32,62.2,3.93104
282754,NONE,EX,RD,D,VS2,EX,EX,14631732,202206,6650.0,0.90,7388.89,6.13,6.18,61.7,3.81306
141550,NONE,EX,RD,J,SI2,EX,EX,13932645,202205,11570.0,1.52,7611.84,7.36,7.39,61.0,4.50790
71237,STG,EX,RD,G,VS1,EX,EX,12699310,202207,8220.0,1.09,7541.28,6.55,6.60,62.0,4.09200


In [ ]:
id_stat = df.groupby(by='id')['carat'].count().to_frame() #Посчитайте статистику по кол-ву камней
id_stat.value_counts()

carat
1        111617
2         41685
3          8688
4           271
5            15
6             2
dtype: int64

In [ ]:
len(id_stat[id_stat.carat == 1])

111617

In [ ]:
id_stat

,carat
id,
0,4
19423,3
19449,2
19476,1
19501,3
...,...
15418480,1
15418482,1
15418486,1


In [ ]:
# разделите выборку на две части
query_id_1 = id_stat[id_stat.carat == 1].index
query_id_1

Int64Index([   19476,    19510,    19613,    19715,    19754,    19776,
               19792,    20580,    20596,    20637,
            ...
            15418458, 15418463, 15418468, 15418472, 15418473, 15418480,
            15418482, 15418486, 15418493, 15418506],
           dtype='int64', name='id', length=111617)

In [ ]:
df_one = df[df.id.isin(query_id_1)] # те камни, которые встречались один раз
df_one

,fluor,symmetry,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y
189202,NONE,EX,PS,D,SI2,NaN,EX,14224443,202205,9120.0,1.51,6039.74,10.67,6.23,59.0,3.67570
76462,NONE,VG,EM,K,SI2,NaN,VG,12902293,202205,1390.0,0.91,1527.47,6.30,4.58,71.0,3.25180
72397,NONE,VG,PS,J,VS1,NaN,VG,12771762,202207,15990.0,2.27,7044.05,11.66,7.26,62.2,4.51572
269475,MED,EX,RD,J,SI2,EX,EX,14584509,202205,3700.0,1.01,3663.37,6.37,6.42,62.0,3.98040
321928,NONE,EX,RD,G,SI1,EX,EX,14773748,202206,7170.0,1.00,7170.00,6.30,6.35,63.7,4.04495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438848,NONE,EX,RD,F,IF,EX,EX,15296246,202207,20250.0,1.30,15576.92,7.03,7.08,60.8,4.30464
387911,NONE,EX,RD,I,VS2,EX,EX,15037555,202206,8980.0,1.35,6651.85,7.05,7.09,61.7,4.37453
165589,STG,G,CUSH,H,SI2,VG,VG,14102502,202205,2970.0,1.01,2940.59,6.12,5.31,71.0,3.77010
141550,NONE,EX,RD,J,SI2,EX,EX,13932645,202205,11570.0,1.52,7611.84,7.36,7.39,61.0,4.50790


In [ ]:
id_stat[id_stat.carat.isin((2, 3))]

,carat
id,
19423,3
19449,2
19501,3
19504,2
19505,2
...,...
15409696,2
15410303,2
15410668,2


In [ ]:
query_id_2_3 = id_stat[id_stat.carat.isin((2, 3))].index
df_three = df[df.id.isin(query_id_2_3)] # те камни, которые встречались 2 или 3 раза
df_three

,fluor,symmetry,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y
135269,NONE,EX,PS,F,SI1,NaN,EX,13870838,202206,12850.0,1.55,8290.32,10.29,6.41,62.0,3.97420
48477,MED,EX,RD,H,VVS2,EX,EX,11725253,202207,7510.0,1.02,7362.75,6.41,6.45,62.5,4.03125
236786,NONE,EX,EM,H,IF,NaN,EX,14444347,202205,21220.0,2.01,10557.21,8.60,6.37,65.0,4.14050
235781,NONE,EX,RD,E,VS2,EX,EX,14438434,202207,8660.0,1.00,8660.00,6.39,6.44,61.4,3.95416
277744,NONE,VG,RD,F,VS1,VG,EX,14615276,202206,8480.0,1.00,8480.00,6.28,6.36,62.3,3.96228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229133,NONE,EX,RD,I,SI2,EX,EX,14409793,202205,3330.0,0.90,3700.00,6.13,6.18,62.0,3.83160
268649,FNT,VG,RD,K,VS1,VG,EX,14581367,202205,3840.0,1.01,3801.98,6.27,6.33,63.0,3.98790
20595,NONE,EX,RD,D,VS2,EX,EX,7535786,202207,8700.0,0.97,8969.07,6.36,6.32,62.2,3.93104
282754,NONE,EX,RD,D,VS2,EX,EX,14631732,202206,6650.0,0.90,7388.89,6.13,6.18,61.7,3.81306


In [ ]:
df_three.sort_values(by=['id','date'])

,fluor,symmetry,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y
5,NONE,EX,RD,D,VVS2,EX,EX,19423,202205,13970.0,1.05,13304.76,6.67,6.64,59.0,3.91760
6,NONE,EX,RD,D,VVS2,EX,EX,19423,202206,13970.0,1.05,13304.76,6.67,6.64,59.7,3.96408
7,NONE,EX,RD,D,VVS2,EX,EX,19423,202207,14550.0,1.05,13857.14,6.67,6.64,59.7,3.96408
8,NONE,EX,RD,D,VVS2,EX,EX,19449,202205,16850.0,1.17,14401.71,6.79,6.75,62.0,4.18500
10,NONE,EX,RD,D,VVS2,EX,EX,19449,202207,17550.0,1.17,15000.00,6.79,6.75,62.0,4.18500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453529,NONE,VG,EM,D,VS1,NaN,EX,15410668,202207,14750.0,1.50,9833.33,7.84,5.43,65.8,3.57294
453543,MED,EX,EM,H,SI2,NaN,EX,15410717,202207,48230.0,4.53,10646.80,11.61,7.57,67.5,5.10975
453544,MED,EX,EM,H,SI2,NaN,EX,15410717,202207,48230.0,4.53,10646.80,11.61,7.57,67.5,5.10975
453566,NONE,EX,PR,G,VVS2,EX,EX,15412954,202207,7100.0,1.20,5916.67,5.80,5.76,72.4,4.17024


In [ ]:
df_three.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109434 entries, 135269 to 71237
Data columns (total 16 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   fluor            109426 non-null  object 
 1   symmetry         109434 non-null  object 
 2   shape            109434 non-null  object 
 3   color            109434 non-null  object 
 4   clarity          109434 non-null  object 
 5   cut              72277 non-null   object 
 6   polish           109434 non-null  object 
 7   id               109434 non-null  int64  
 8   date             109434 non-null  int64  
 9   price            109434 non-null  float64
 10  carat            109434 non-null  float64
 11  price_per_carat  109434 non-null  float64
 12  z                109434 non-null  float64
 13  x                109434 non-null  float64
 14  depth_perc       109434 non-null  float64
 15  y                109434 non-null  float64
dtypes: float64(7), int64(2), object(7)

In [ ]:
# Проверим нулевые значения в колонке fluor            
df_three[df_three.fluor.isnull()]

,fluor,symmetry,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y
93957,NaN,VG,OV,H,SI2,NaN,VG,13262095,202205,4170.0,0.97,4298.97,7.75,5.42,63.0,3.41460
102938,NaN,VG,PS,G,SI2,NaN,VG,13412217,202207,4630.0,0.96,4822.92,8.60,5.51,61.3,3.37763
93960,NaN,VG,OV,D,SI2,NaN,EX,13262100,202205,5080.0,0.96,5291.67,7.96,5.35,62.0,3.31700
94123,NaN,EX,EM,F,SI1,NaN,EX,13264995,202205,14480.0,1.70,8517.65,7.87,5.60,68.0,3.80800
102937,NaN,VG,PS,G,SI2,NaN,VG,13412217,202206,4450.0,0.96,4635.42,8.60,5.51,61.3,3.37763
94125,NaN,EX,EM,F,SI1,NaN,EX,13264995,202207,14790.0,1.70,8700.00,7.87,5.60,68.1,3.81360
93959,NaN,VG,OV,H,SI2,NaN,VG,13262095,202207,4340.0,0.97,4474.23,7.75,5.42,63.3,3.43086
93962,NaN,VG,OV,D,SI2,NaN,EX,13262100,202207,5290.0,0.96,5510.42,7.96,5.35,62.8,3.35980


In [ ]:
df_three[df_three.fluor.isnull()].id.values

array([13262095, 13412217, 13262100, 13264995, 13412217, 13264995,
       13262095, 13262100])

In [ ]:
query_id_flour_na = df_three[df_three.fluor.isnull()].id.values
df_three[df_three.id.isin(query_id_flour_na)].sort_values(by=['id','date'])


,fluor,symmetry,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y
93957,NaN,VG,OV,H,SI2,NaN,VG,13262095,202205,4170.0,0.97,4298.97,7.75,5.42,63.0,3.41460
93959,NaN,VG,OV,H,SI2,NaN,VG,13262095,202207,4340.0,0.97,4474.23,7.75,5.42,63.3,3.43086
93960,NaN,VG,OV,D,SI2,NaN,EX,13262100,202205,5080.0,0.96,5291.67,7.96,5.35,62.0,3.31700
93962,NaN,VG,OV,D,SI2,NaN,EX,13262100,202207,5290.0,0.96,5510.42,7.96,5.35,62.8,3.35980
94123,NaN,EX,EM,F,SI1,NaN,EX,13264995,202205,14480.0,1.70,8517.65,7.87,5.60,68.0,3.80800
94125,NaN,EX,EM,F,SI1,NaN,EX,13264995,202207,14790.0,1.70,8700.00,7.87,5.60,68.1,3.81360
102937,NaN,VG,PS,G,SI2,NaN,VG,13412217,202206,4450.0,0.96,4635.42,8.60,5.51,61.3,3.37763
102938,NaN,VG,PS,G,SI2,NaN,VG,13412217,202207,4630.0,0.96,4822.92,8.60,5.51,61.3,3.37763


**Вывод:** в колонке flour по значению id не удалось найти недостающие данные.

Удалим строки с пустыми значения в колонке flour.
Примечание. Как вариант попробовать предсказать значения.


In [ ]:
df_three = df_three[df_three.fluor.notnull()]
df_three.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109426 entries, 135269 to 71237
Data columns (total 16 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   fluor            109426 non-null  object 
 1   symmetry         109426 non-null  object 
 2   shape            109426 non-null  object 
 3   color            109426 non-null  object 
 4   clarity          109426 non-null  object 
 5   cut              72277 non-null   object 
 6   polish           109426 non-null  object 
 7   id               109426 non-null  int64  
 8   date             109426 non-null  int64  
 9   price            109426 non-null  float64
 10  carat            109426 non-null  float64
 11  price_per_carat  109426 non-null  float64
 12  z                109426 non-null  float64
 13  x                109426 non-null  float64
 14  depth_perc       109426 non-null  float64
 15  y                109426 non-null  float64
dtypes: float64(7), int64(2), object(7)

In [ ]:
# Уникальные значения в колонке fluor
df_three.fluor.unique()

array(['NONE', 'MED', 'FNT', 'STG'], dtype=object)

In [ ]:
# Посмотрим на столбец cut
df_three.cut.unique()

array([nan, 'EX', 'VG', 'G'], dtype=object)

In [ ]:
# Посмотрим на столбец shape
df_three['shape'].unique()

array(['PS', 'RD', 'EM', 'RAD', 'OV', 'CUSH', 'PR', 'HS', 'EM SQ', 'MQ'],
      dtype=object)

In [ ]:
df_three[df_three.cut.notnull()][['shape', 'cut']]['shape'].unique()

array(['RD', 'CUSH', 'PR'], dtype=object)

In [ ]:
df_three.groupby('shape').cut.count()

shape
CUSH      7438
EM           0
EM SQ        0
HS           0
MQ           0
OV           0
PR        7510
PS           0
RAD          0
RD       57329
Name: cut, dtype: int64

Вывод: наличие пустых значений в cut допустимо, тк 'cut - качество огранки (может быть только у круглых камней)'

Хотя значения cut есть не только у круглых камней.

In [ ]:
df_three.describe()

,id,date,price,carat,price_per_carat,z,x,depth_perc,y
count,1.094260e+05,109426.000000,1.094260e+05,109426.000000,109426.000000,109426.000000,109426.000000,109426.000000,109426.000000
mean,1.319217e+07,202206.045483,1.204960e+04,1.343608,7046.232714,7.061422,6.408138,64.167445,4.093460
std,2.839245e+06,0.805910,2.885300e+04,0.768056,5113.156695,1.399606,1.069207,4.272113,0.611992
min,1.942300e+04,202205.000000,9.800000e+02,0.810000,1195.120000,4.840000,3.870000,46.000000,2.559300
25%,1.318985e+07,202205.000000,4.130000e+03,1.000000,4091.670000,6.240000,5.660000,61.800000,3.801600
50%,1.420679e+07,202206.000000,6.430000e+03,1.010000,5798.125000,6.510000,6.380000,63.000000,3.993600
75%,1.466596e+07,202207.000000,1.137000e+04,1.500000,8566.670000,7.600000,6.780000,66.500000,4.291677
max,1.541295e+07,202207.000000,2.380380e+06,21.170000,165340.000000,21.560000,18.050000,78.000000,10.739750


In [ ]:
df_three = df_three.groupby('id').agg('last').reset_index() #тепреь в качестве цены возьмем последнее значение по времени
df_three

,id,fluor,symmetry,shape,color,clarity,cut,polish,date,price,carat,price_per_carat,z,x,depth_perc,y
0,19423,NONE,EX,RD,D,VVS2,EX,EX,202206,13970.0,1.05,13304.76,6.67,6.64,59.7,3.96408
1,19449,NONE,EX,RD,D,VVS2,EX,EX,202207,17550.0,1.17,15000.00,6.79,6.75,62.0,4.18500
2,19501,FNT,VG,RD,D,VS1,VG,G,202207,36420.0,2.00,18210.00,8.01,7.92,62.6,4.95792
3,19504,NONE,VG,EM SQ,E,VS2,None,EX,202206,23140.0,2.01,11512.44,7.03,6.70,73.3,4.91110
4,19505,FNT,VG,RD,I,VVS1,EX,EX,202205,27980.0,2.01,13920.40,8.01,8.09,62.0,5.01580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50364,15409696,NONE,EX,RD,I,SI1,EX,EX,202207,6040.0,1.01,5980.20,6.46,6.49,60.8,3.94592
50365,15410303,MED,EX,OV,H,VVS2,None,EX,202207,6620.0,1.01,6554.46,7.76,5.40,65.0,3.51000
50366,15410668,NONE,VG,EM,D,VS1,None,EX,202207,14750.0,1.50,9833.33,7.84,5.43,65.8,3.57294
50367,15410717,MED,EX,EM,H,SI2,None,EX,202207,48230.0,4.53,10646.80,11.61,7.57,67.5,5.10975


In [ ]:
df_three

,id,fluor,symmetry,shape,color,clarity,cut,polish,date,price,carat,price_per_carat,z,x,depth_perc,y
0,19423,NONE,EX,RD,D,VVS2,EX,EX,202206,13970.0,1.05,13304.76,6.67,6.64,59.7,3.96408
1,19449,NONE,EX,RD,D,VVS2,EX,EX,202207,17550.0,1.17,15000.00,6.79,6.75,62.0,4.18500
2,19501,FNT,VG,RD,D,VS1,VG,G,202207,36420.0,2.00,18210.00,8.01,7.92,62.6,4.95792
3,19504,NONE,VG,EM SQ,E,VS2,None,EX,202206,23140.0,2.01,11512.44,7.03,6.70,73.3,4.91110
4,19505,FNT,VG,RD,I,VVS1,EX,EX,202205,27980.0,2.01,13920.40,8.01,8.09,62.0,5.01580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50364,15409696,NONE,EX,RD,I,SI1,EX,EX,202207,6040.0,1.01,5980.20,6.46,6.49,60.8,3.94592
50365,15410303,MED,EX,OV,H,VVS2,None,EX,202207,6620.0,1.01,6554.46,7.76,5.40,65.0,3.51000
50366,15410668,NONE,VG,EM,D,VS1,None,EX,202207,14750.0,1.50,9833.33,7.84,5.43,65.8,3.57294
50367,15410717,MED,EX,EM,H,SI2,None,EX,202207,48230.0,4.53,10646.80,11.61,7.57,67.5,5.10975


In [ ]:
df = pd.concat([df_one, df_three]).reset_index(drop=True) # соединяем результаты
df

,fluor,symmetry,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y
0,NONE,EX,PS,D,SI2,NaN,EX,14224443,202205,9120.0,1.51,6039.74,10.67,6.23,59.0,3.67570
1,NONE,VG,EM,K,SI2,NaN,VG,12902293,202205,1390.0,0.91,1527.47,6.30,4.58,71.0,3.25180
2,NONE,VG,PS,J,VS1,NaN,VG,12771762,202207,15990.0,2.27,7044.05,11.66,7.26,62.2,4.51572
3,MED,EX,RD,J,SI2,EX,EX,14584509,202205,3700.0,1.01,3663.37,6.37,6.42,62.0,3.98040
4,NONE,EX,RD,G,SI1,EX,EX,14773748,202206,7170.0,1.00,7170.00,6.30,6.35,63.7,4.04495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161981,NONE,EX,RD,I,SI1,EX,EX,15409696,202207,6040.0,1.01,5980.20,6.46,6.49,60.8,3.94592
161982,MED,EX,OV,H,VVS2,None,EX,15410303,202207,6620.0,1.01,6554.46,7.76,5.40,65.0,3.51000
161983,NONE,VG,EM,D,VS1,None,EX,15410668,202207,14750.0,1.50,9833.33,7.84,5.43,65.8,3.57294
161984,MED,EX,EM,H,SI2,None,EX,15410717,202207,48230.0,4.53,10646.80,11.61,7.57,67.5,5.10975


In [ ]:
# Посчитайте стандартное отклонение по нескольким полям,
stat = df.describe()
stat

,id,date,price,carat,price_per_carat,z,x,depth_perc,y
count,1.619860e+05,161986.000000,1.619860e+05,161986.000000,161986.000000,161986.000000,161986.000000,161986.000000,161986.000000
mean,1.374806e+07,202206.040880,1.142693e+04,1.338462,7010.592296,7.085985,6.427245,64.056342,4.099810
std,2.401233e+06,0.830156,2.271707e+04,0.688466,5185.008479,1.354897,1.030178,4.158366,0.589525
min,1.942300e+04,202205.000000,9.200000e+02,0.810000,1029.700000,4.520000,3.870000,46.000000,2.556000
25%,1.375422e+07,202205.000000,4.240000e+03,1.000000,4170.000000,6.260000,5.700000,61.700000,3.814050
50%,1.447881e+07,202206.000000,6.610000e+03,1.010000,5872.950000,6.540000,6.380000,62.900000,3.999460
75%,1.490779e+07,202207.000000,1.165000e+04,1.500000,8613.330000,7.670000,6.830000,66.000000,4.333000
max,1.541851e+07,202207.000000,2.286110e+06,21.170000,955000.000000,21.880000,18.050000,78.000000,10.739750


In [ ]:
df.dtypes

fluor               object
symmetry            object
shape               object
color               object
clarity             object
cut                 object
polish              object
id                   int64
date                 int64
price              float64
carat              float64
price_per_carat    float64
z                  float64
x                  float64
depth_perc         float64
y                  float64
dtype: object

In [ ]:
# Сохраним копию до удаления выбросов
df_with_outliers = df.copy()
df_with_outliers

,fluor,symmetry,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y
0,NONE,EX,PS,D,SI2,NaN,EX,14224443,202205,9120.0,1.51,6039.74,10.67,6.23,59.0,3.67570
1,NONE,VG,EM,K,SI2,NaN,VG,12902293,202205,1390.0,0.91,1527.47,6.30,4.58,71.0,3.25180
2,NONE,VG,PS,J,VS1,NaN,VG,12771762,202207,15990.0,2.27,7044.05,11.66,7.26,62.2,4.51572
3,MED,EX,RD,J,SI2,EX,EX,14584509,202205,3700.0,1.01,3663.37,6.37,6.42,62.0,3.98040
4,NONE,EX,RD,G,SI1,EX,EX,14773748,202206,7170.0,1.00,7170.00,6.30,6.35,63.7,4.04495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161981,NONE,EX,RD,I,SI1,EX,EX,15409696,202207,6040.0,1.01,5980.20,6.46,6.49,60.8,3.94592
161982,MED,EX,OV,H,VVS2,None,EX,15410303,202207,6620.0,1.01,6554.46,7.76,5.40,65.0,3.51000
161983,NONE,VG,EM,D,VS1,None,EX,15410668,202207,14750.0,1.50,9833.33,7.84,5.43,65.8,3.57294
161984,MED,EX,EM,H,SI2,None,EX,15410717,202207,48230.0,4.53,10646.80,11.61,7.57,67.5,5.10975


In [ ]:
#  Выбросы — это те значения данных, которые находятся за пределами следующих интервалов: Q1–1.5 x IQR и Q3 + 1.5 x IQR.
def outlier_params(s):
    Q1 = s.describe()['25%']
    Q3 = s.describe()['75%']
    iqr = Q3 - Q1
    return Q1 - 1.5 * iqr, Q3 + 1.5 * iqr


In [ ]:
# Удалите аномальные наблюдения

num_params = ['date', 'price', 'carat', 'price_per_carat', 'z', 'x', 'depth_perc', 'y']
outliers = {}

# Сохраним в словарь граничные параметры выбросов
for param in num_params:
    buttom, top = outlier_params(df[param])
    outliers[param] = {
        'outlier_buttom': buttom,
        'outlier_top': top        
    }

# Удалим выбросы
print('before outlier remove ', len(df))
for param in num_params:
    buttom = outliers[param]['outlier_buttom']
    top = outliers[param]['outlier_top']
    
    df = df[(buttom <= df[param]) & (df[param] <= top)]
        
print('after outlier remove ', len(df))
df

before outlier remove  161986
after outlier remove  130234


,fluor,symmetry,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y
1,NONE,VG,EM,K,SI2,NaN,VG,12902293,202205,1390.0,0.91,1527.47,6.30,4.58,71.0,3.25180
3,MED,EX,RD,J,SI2,EX,EX,14584509,202205,3700.0,1.01,3663.37,6.37,6.42,62.0,3.98040
4,NONE,EX,RD,G,SI1,EX,EX,14773748,202206,7170.0,1.00,7170.00,6.30,6.35,63.7,4.04495
5,MED,EX,OV,H,VS2,NaN,EX,15281430,202207,5160.0,1.00,5160.00,7.43,5.64,64.6,3.64344
6,NONE,VG,OV,G,SI1,NaN,EX,14844572,202205,12070.0,1.51,7993.38,9.41,6.42,57.0,3.65940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161980,NONE,EX,RD,K,SI1,EX,EX,15409661,202207,4300.0,1.08,3981.48,6.55,6.56,62.1,4.07376
161981,NONE,EX,RD,I,SI1,EX,EX,15409696,202207,6040.0,1.01,5980.20,6.46,6.49,60.8,3.94592
161982,MED,EX,OV,H,VVS2,None,EX,15410303,202207,6620.0,1.01,6554.46,7.76,5.40,65.0,3.51000
161983,NONE,VG,EM,D,VS1,None,EX,15410668,202207,14750.0,1.50,9833.33,7.84,5.43,65.8,3.57294


Цены на бриллианты достаточно сильно меняются, попробуйте вычислить коэффициент инфляции и привести цены к последнему месяцу. Стоит учесть, что цены на все бриллианты не изменяются синхронно, то есть изменение в определенных группах может быть разным.

Определите эти группы и рассчитайте коэффициенты инфляции для каждой из них. Подправьте цены на эти коэффициенты и создайте новую переменную.

In [ ]:
categories = ['fluor', 'symmetry', 'shape', 'color', 'clarity', 'cut', 'polish'] #определите по каким группам отпределять инфляцию (к примеру можно добавить каратную группу)

df_index = df.groupby(['date'] + categories)[['price_per_carat']]\
                                                                .mean()\
                                                                    .reset_index() # индекс цен
df_index

,date,fluor,symmetry,shape,color,clarity,cut,polish,price_per_carat
0,202205,FNT,EX,CUSH,D,IF,G,EX,6840.000000
1,202205,FNT,EX,CUSH,D,SI1,EX,EX,6204.078000
2,202205,FNT,EX,CUSH,D,SI1,G,EX,2802.200000
3,202205,FNT,EX,CUSH,D,SI1,VG,EX,5688.955000
4,202205,FNT,EX,CUSH,D,SI2,EX,EX,3493.530000
...,...,...,...,...,...,...,...,...,...
10640,202207,STG,VG,RD,K,VS2,VG,VG,3737.436667
10641,202207,STG,VG,RD,K,VVS1,VG,VG,5840.000000
10642,202207,STG,VG,RD,K,VVS2,EX,EX,7215.000000
10643,202207,STG,VG,RD,K,VVS2,VG,EX,3311.110000


In [ ]:
date_max = df_index.date.max()
date_max

202207

In [ ]:
df_tmp = df_index.query('date == @date_max')[categories + ['price_per_carat']].rename(columns={'price_per_carat': 'price_per_carat_max'})
df_tmp

,fluor,symmetry,shape,color,clarity,cut,polish,price_per_carat_max
7456,FNT,EX,CUSH,D,SI1,EX,EX,8897.125000
7457,FNT,EX,CUSH,D,SI1,EX,VG,6900.000000
7458,FNT,EX,CUSH,D,SI1,VG,EX,6264.876667
7459,FNT,EX,CUSH,D,SI2,VG,EX,6517.613333
7460,FNT,EX,CUSH,D,VS1,EX,EX,7336.540000
...,...,...,...,...,...,...,...,...
10640,STG,VG,RD,K,VS2,VG,VG,3737.436667
10641,STG,VG,RD,K,VVS1,VG,VG,5840.000000
10642,STG,VG,RD,K,VVS2,EX,EX,7215.000000
10643,STG,VG,RD,K,VVS2,VG,EX,3311.110000


In [ ]:
df_index = df_index.merge(df_tmp, on=categories, how='outer') # сопоставляем группы с максимальной датой
df_index

,date,fluor,symmetry,shape,color,clarity,cut,polish,price_per_carat,price_per_carat_max
0,202205,FNT,EX,CUSH,D,IF,G,EX,6840.000,NaN
1,202205,FNT,EX,CUSH,D,SI1,EX,EX,6204.078,8897.125
2,202206,FNT,EX,CUSH,D,SI1,EX,EX,4568.940,8897.125
3,202207,FNT,EX,CUSH,D,SI1,EX,EX,8897.125,8897.125
4,202205,FNT,EX,CUSH,D,SI1,G,EX,2802.200,NaN
...,...,...,...,...,...,...,...,...,...,...
10640,202207,STG,VG,RD,K,I1,G,EX,3220.000,3220.000
10641,202207,STG,VG,RD,K,IF,VG,EX,4431.370,4431.370
10642,202207,STG,VG,RD,K,SI2,G,EX,2600.000,2600.000
10643,202207,STG,VG,RD,K,VS2,EX,EX,3640.000,3640.000


In [ ]:
df_index[(df_index["fluor"]=='FNT') &	(df_index["symmetry"]=='EX') &	(df_index["shape"]=='CUSH') &	(df_index["color"]=='D') &	(df_index["clarity"]=='SI1') &	(df_index["cut"]=='EX') & (df_index["polish"]=='EX')]

,date,fluor,symmetry,shape,color,clarity,cut,polish,price_per_carat,price_per_carat_max
1,202205,FNT,EX,CUSH,D,SI1,EX,EX,6204.078,8897.125
2,202206,FNT,EX,CUSH,D,SI1,EX,EX,4568.940,8897.125
3,202207,FNT,EX,CUSH,D,SI1,EX,EX,8897.125,8897.125


In [ ]:
df_index[(df_index["fluor"]=='FNT') &	(df_index["symmetry"]=='EX') &	(df_index["shape"]=='CUSH') &	(df_index["color"]=='D') &	(df_index["clarity"]=='SI2') &	(df_index["cut"]=='EX') & (df_index["polish"]=='EX')]


,date,fluor,symmetry,shape,color,clarity,cut,polish,price_per_carat,price_per_carat_max
8,202205,FNT,EX,CUSH,D,SI2,EX,EX,3493.53,NaN
9,202206,FNT,EX,CUSH,D,SI2,EX,EX,5512.95,NaN


In [ ]:
df_index[df_index.price_per_carat_max.isna()]

,date,fluor,symmetry,shape,color,clarity,cut,polish,price_per_carat,price_per_carat_max
0,202205,FNT,EX,CUSH,D,IF,G,EX,6840.000,NaN
4,202205,FNT,EX,CUSH,D,SI1,G,EX,2802.200,NaN
8,202205,FNT,EX,CUSH,D,SI2,EX,EX,3493.530,NaN
9,202206,FNT,EX,CUSH,D,SI2,EX,EX,5512.950,NaN
10,202205,FNT,EX,CUSH,D,SI2,G,EX,6900.500,NaN
...,...,...,...,...,...,...,...,...,...,...
10044,202206,STG,VG,RD,K,I1,VG,EX,3220.000,NaN
10045,202206,STG,VG,RD,K,SI1,EX,VG,2588.890,NaN
10046,202206,STG,VG,RD,K,SI1,G,EX,4760.000,NaN
10047,202206,STG,VG,RD,K,SI2,EX,VG,5053.330,NaN


In [ ]:
df_index[df_index.price_per_carat_max.isna()]

,date,fluor,symmetry,shape,color,clarity,cut,polish,price_per_carat,price_per_carat_max
0,202205,FNT,EX,CUSH,D,IF,G,EX,6840.000,NaN
4,202205,FNT,EX,CUSH,D,SI1,G,EX,2802.200,NaN
8,202205,FNT,EX,CUSH,D,SI2,EX,EX,3493.530,NaN
9,202206,FNT,EX,CUSH,D,SI2,EX,EX,5512.950,NaN
10,202205,FNT,EX,CUSH,D,SI2,G,EX,6900.500,NaN
...,...,...,...,...,...,...,...,...,...,...
10044,202206,STG,VG,RD,K,I1,VG,EX,3220.000,NaN
10045,202206,STG,VG,RD,K,SI1,EX,VG,2588.890,NaN
10046,202206,STG,VG,RD,K,SI1,G,EX,4760.000,NaN
10047,202206,STG,VG,RD,K,SI2,EX,VG,5053.330,NaN


In [ ]:
df_index['inflation'] = df_index.dropna()['price_per_carat_max'] / df_index.price_per_carat # вычисляем инфляцию
df_index

,date,fluor,symmetry,shape,color,clarity,cut,polish,price_per_carat,price_per_carat_max,inflation
0,202205,FNT,EX,CUSH,D,IF,G,EX,6840.000,NaN,NaN
1,202205,FNT,EX,CUSH,D,SI1,EX,EX,6204.078,8897.125,1.434077
2,202206,FNT,EX,CUSH,D,SI1,EX,EX,4568.940,8897.125,1.947306
3,202207,FNT,EX,CUSH,D,SI1,EX,EX,8897.125,8897.125,1.000000
4,202205,FNT,EX,CUSH,D,SI1,G,EX,2802.200,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
10640,202207,STG,VG,RD,K,I1,G,EX,3220.000,3220.000,1.000000
10641,202207,STG,VG,RD,K,IF,VG,EX,4431.370,4431.370,1.000000
10642,202207,STG,VG,RD,K,SI2,G,EX,2600.000,2600.000,1.000000
10643,202207,STG,VG,RD,K,VS2,EX,EX,3640.000,3640.000,1.000000


In [ ]:
df_with_inf = df.merge(df_index[['date'] + categories + ['inflation']], 
                        on=['date'] + categories, how='left') # соединяем все в одной таблице
df_with_inf

,fluor,symmetry,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y,inflation
0,NONE,VG,EM,K,SI2,NaN,VG,12902293,202205,1390.0,0.91,1527.47,6.30,4.58,71.0,3.25180,NaN
1,MED,EX,RD,J,SI2,EX,EX,14584509,202205,3700.0,1.01,3663.37,6.37,6.42,62.0,3.98040,1.003894
2,NONE,EX,RD,G,SI1,EX,EX,14773748,202206,7170.0,1.00,7170.00,6.30,6.35,63.7,4.04495,1.013707
3,MED,EX,OV,H,VS2,NaN,EX,15281430,202207,5160.0,1.00,5160.00,7.43,5.64,64.6,3.64344,NaN
4,NONE,VG,OV,G,SI1,NaN,EX,14844572,202205,12070.0,1.51,7993.38,9.41,6.42,57.0,3.65940,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130229,NONE,EX,RD,K,SI1,EX,EX,15409661,202207,4300.0,1.08,3981.48,6.55,6.56,62.1,4.07376,1.000000
130230,NONE,EX,RD,I,SI1,EX,EX,15409696,202207,6040.0,1.01,5980.20,6.46,6.49,60.8,3.94592,1.000000
130231,MED,EX,OV,H,VVS2,None,EX,15410303,202207,6620.0,1.01,6554.46,7.76,5.40,65.0,3.51000,NaN
130232,NONE,VG,EM,D,VS1,None,EX,15410668,202207,14750.0,1.50,9833.33,7.84,5.43,65.8,3.57294,NaN


In [ ]:
# Посмотрим количество пустых ячеек инфляции
df_with_inf.inflation.isna().sum()

44349

In [ ]:
# Выберем другие параметры для вычисления инфляции - удалим 'color', 'clarity' и 'fluor'

categories = ['symmetry', 'shape', 'cut', 'polish'] #определите по каким группам отпределять инфляцию (к примеру можно добавить каратную группу)

df_index = df.groupby(['date'] + categories)[['price_per_carat']]\
                                                                .mean()\
                                                                    .reset_index() # индекс цен

date_max = df_index.date.max()
df_tmp = df_index.query('date == @date_max')[categories + ['price_per_carat']].rename(columns={'price_per_carat': 'price_per_carat_max'})
df_index = df_index.merge(df_tmp, on=categories, how='outer') # сопоставляем группы с максимальной датой


df_index['inflation_2cat_group'] = df_index.dropna()['price_per_carat_max'] / df_index.price_per_carat # вычисляем инфляцию

df_with_inf = df_with_inf.merge(df_index[['date'] + categories + ['inflation_2cat_group']], 
                        on=['date'] + categories, how='left') # соединяем все в одной таблице
df_with_inf

,fluor,symmetry,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y,inflation,inflation_2cat_group
0,NONE,VG,EM,K,SI2,NaN,VG,12902293,202205,1390.0,0.91,1527.47,6.30,4.58,71.0,3.25180,NaN,NaN
1,MED,EX,RD,J,SI2,EX,EX,14584509,202205,3700.0,1.01,3663.37,6.37,6.42,62.0,3.98040,1.003894,1.015418
2,NONE,EX,RD,G,SI1,EX,EX,14773748,202206,7170.0,1.00,7170.00,6.30,6.35,63.7,4.04495,1.013707,1.008800
3,MED,EX,OV,H,VS2,NaN,EX,15281430,202207,5160.0,1.00,5160.00,7.43,5.64,64.6,3.64344,NaN,NaN
4,NONE,VG,OV,G,SI1,NaN,EX,14844572,202205,12070.0,1.51,7993.38,9.41,6.42,57.0,3.65940,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130229,NONE,EX,RD,K,SI1,EX,EX,15409661,202207,4300.0,1.08,3981.48,6.55,6.56,62.1,4.07376,1.000000,1.000000
130230,NONE,EX,RD,I,SI1,EX,EX,15409696,202207,6040.0,1.01,5980.20,6.46,6.49,60.8,3.94592,1.000000,1.000000
130231,MED,EX,OV,H,VVS2,None,EX,15410303,202207,6620.0,1.01,6554.46,7.76,5.40,65.0,3.51000,NaN,NaN
130232,NONE,VG,EM,D,VS1,None,EX,15410668,202207,14750.0,1.50,9833.33,7.84,5.43,65.8,3.57294,NaN,NaN


In [ ]:
df_with_inf.inflation_2cat_group.isna().sum()

39599

In [ ]:
# Выберем другие параметры для вычисления инфляции - удалим 'symmetry', 'polish'

categories = [ 'shape', 'cut'] #определите по каким группам отпределять инфляцию (к примеру можно добавить каратную группу)

df_index = df.groupby(['date'] + categories)[['price_per_carat']]\
                                                                .mean()\
                                                                    .reset_index() # индекс цен

date_max = df_index.date.max()
df_tmp = df_index.query('date == @date_max')[categories + ['price_per_carat']].rename(columns={'price_per_carat': 'price_per_carat_max'})
df_index = df_index.merge(df_tmp, on=categories, how='outer') # сопоставляем группы с максимальной датой


df_index['inflation_3cat_group'] = df_index.dropna()['price_per_carat_max'] / df_index.price_per_carat # вычисляем инфляцию

df_with_inf = df_with_inf.merge(df_index[['date'] + categories + ['inflation_3cat_group']], 
                        on=['date'] + categories, how='left') # соединяем все в одной таблице
df_with_inf

,fluor,symmetry,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y,inflation,inflation_2cat_group,inflation_3cat_group
0,NONE,VG,EM,K,SI2,NaN,VG,12902293,202205,1390.0,0.91,1527.47,6.30,4.58,71.0,3.25180,NaN,NaN,NaN
1,MED,EX,RD,J,SI2,EX,EX,14584509,202205,3700.0,1.01,3663.37,6.37,6.42,62.0,3.98040,1.003894,1.015418,1.013542
2,NONE,EX,RD,G,SI1,EX,EX,14773748,202206,7170.0,1.00,7170.00,6.30,6.35,63.7,4.04495,1.013707,1.008800,1.009436
3,MED,EX,OV,H,VS2,NaN,EX,15281430,202207,5160.0,1.00,5160.00,7.43,5.64,64.6,3.64344,NaN,NaN,NaN
4,NONE,VG,OV,G,SI1,NaN,EX,14844572,202205,12070.0,1.51,7993.38,9.41,6.42,57.0,3.65940,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130229,NONE,EX,RD,K,SI1,EX,EX,15409661,202207,4300.0,1.08,3981.48,6.55,6.56,62.1,4.07376,1.000000,1.000000,1.000000
130230,NONE,EX,RD,I,SI1,EX,EX,15409696,202207,6040.0,1.01,5980.20,6.46,6.49,60.8,3.94592,1.000000,1.000000,1.000000
130231,MED,EX,OV,H,VVS2,None,EX,15410303,202207,6620.0,1.01,6554.46,7.76,5.40,65.0,3.51000,NaN,NaN,NaN
130232,NONE,VG,EM,D,VS1,None,EX,15410668,202207,14750.0,1.50,9833.33,7.84,5.43,65.8,3.57294,NaN,NaN,NaN


In [ ]:
df_with_inf.inflation_3cat_group.isna().sum()

39591

In [ ]:
# Выберем другие параметры для вычисления инфляции - удалим cut

categories = [ 'shape'] #определите по каким группам отпределять инфляцию (к примеру можно добавить каратную группу)

df_index = df.groupby(['date'] + categories)[['price_per_carat']]\
                                                                .mean()\
                                                                    .reset_index() # индекс цен

date_max = df_index.date.max()
df_tmp = df_index.query('date == @date_max')[categories + ['price_per_carat']].rename(columns={'price_per_carat': 'price_per_carat_max'})
df_index = df_index.merge(df_tmp, on=categories, how='outer') # сопоставляем группы с максимальной датой


df_index['inflation_4cat_group'] = df_index.dropna()['price_per_carat_max'] / df_index.price_per_carat # вычисляем инфляцию

df_with_inf = df_with_inf.merge(df_index[['date'] + categories + ['inflation_4cat_group']], 
                        on=['date'] + categories, how='left') # соединяем все в одной таблице
df_with_inf

,fluor,symmetry,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y,inflation,inflation_2cat_group,inflation_3cat_group,inflation_4cat_group
0,NONE,VG,EM,K,SI2,NaN,VG,12902293,202205,1390.0,0.91,1527.47,6.30,4.58,71.0,3.25180,NaN,NaN,NaN,1.037988
1,MED,EX,RD,J,SI2,EX,EX,14584509,202205,3700.0,1.01,3663.37,6.37,6.42,62.0,3.98040,1.003894,1.015418,1.013542,1.015676
2,NONE,EX,RD,G,SI1,EX,EX,14773748,202206,7170.0,1.00,7170.00,6.30,6.35,63.7,4.04495,1.013707,1.008800,1.009436,1.010882
3,MED,EX,OV,H,VS2,NaN,EX,15281430,202207,5160.0,1.00,5160.00,7.43,5.64,64.6,3.64344,NaN,NaN,NaN,1.000000
4,NONE,VG,OV,G,SI1,NaN,EX,14844572,202205,12070.0,1.51,7993.38,9.41,6.42,57.0,3.65940,NaN,NaN,NaN,1.019546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130229,NONE,EX,RD,K,SI1,EX,EX,15409661,202207,4300.0,1.08,3981.48,6.55,6.56,62.1,4.07376,1.000000,1.000000,1.000000,1.000000
130230,NONE,EX,RD,I,SI1,EX,EX,15409696,202207,6040.0,1.01,5980.20,6.46,6.49,60.8,3.94592,1.000000,1.000000,1.000000,1.000000
130231,MED,EX,OV,H,VVS2,None,EX,15410303,202207,6620.0,1.01,6554.46,7.76,5.40,65.0,3.51000,NaN,NaN,NaN,1.000000
130232,NONE,VG,EM,D,VS1,None,EX,15410668,202207,14750.0,1.50,9833.33,7.84,5.43,65.8,3.57294,NaN,NaN,NaN,1.000000


In [ ]:
df_with_inf.inflation_4cat_group.isna().sum()

0

In [ ]:
# Заполним пустые инфляции из более широкой группы
df_with_inf.inflation_3cat_group.fillna(df_with_inf.inflation_4cat_group[df_with_inf.inflation_3cat_group.isna() == True], inplace=True)
df_with_inf

,fluor,symmetry,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y,inflation,inflation_2cat_group,inflation_3cat_group,inflation_4cat_group
0,NONE,VG,EM,K,SI2,NaN,VG,12902293,202205,1390.0,0.91,1527.47,6.30,4.58,71.0,3.25180,NaN,NaN,1.037988,1.037988
1,MED,EX,RD,J,SI2,EX,EX,14584509,202205,3700.0,1.01,3663.37,6.37,6.42,62.0,3.98040,1.003894,1.015418,1.013542,1.015676
2,NONE,EX,RD,G,SI1,EX,EX,14773748,202206,7170.0,1.00,7170.00,6.30,6.35,63.7,4.04495,1.013707,1.008800,1.009436,1.010882
3,MED,EX,OV,H,VS2,NaN,EX,15281430,202207,5160.0,1.00,5160.00,7.43,5.64,64.6,3.64344,NaN,NaN,1.000000,1.000000
4,NONE,VG,OV,G,SI1,NaN,EX,14844572,202205,12070.0,1.51,7993.38,9.41,6.42,57.0,3.65940,NaN,NaN,1.019546,1.019546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130229,NONE,EX,RD,K,SI1,EX,EX,15409661,202207,4300.0,1.08,3981.48,6.55,6.56,62.1,4.07376,1.000000,1.000000,1.000000,1.000000
130230,NONE,EX,RD,I,SI1,EX,EX,15409696,202207,6040.0,1.01,5980.20,6.46,6.49,60.8,3.94592,1.000000,1.000000,1.000000,1.000000
130231,MED,EX,OV,H,VVS2,None,EX,15410303,202207,6620.0,1.01,6554.46,7.76,5.40,65.0,3.51000,NaN,NaN,1.000000,1.000000
130232,NONE,VG,EM,D,VS1,None,EX,15410668,202207,14750.0,1.50,9833.33,7.84,5.43,65.8,3.57294,NaN,NaN,1.000000,1.000000


In [ ]:
df_with_inf.inflation_2cat_group.fillna(df_with_inf.inflation_3cat_group[df_with_inf.inflation_2cat_group.isna() == True], inplace=True)
df_with_inf

,fluor,symmetry,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y,inflation,inflation_2cat_group,inflation_3cat_group,inflation_4cat_group
0,NONE,VG,EM,K,SI2,NaN,VG,12902293,202205,1390.0,0.91,1527.47,6.30,4.58,71.0,3.25180,NaN,1.037988,1.037988,1.037988
1,MED,EX,RD,J,SI2,EX,EX,14584509,202205,3700.0,1.01,3663.37,6.37,6.42,62.0,3.98040,1.003894,1.015418,1.013542,1.015676
2,NONE,EX,RD,G,SI1,EX,EX,14773748,202206,7170.0,1.00,7170.00,6.30,6.35,63.7,4.04495,1.013707,1.008800,1.009436,1.010882
3,MED,EX,OV,H,VS2,NaN,EX,15281430,202207,5160.0,1.00,5160.00,7.43,5.64,64.6,3.64344,NaN,1.000000,1.000000,1.000000
4,NONE,VG,OV,G,SI1,NaN,EX,14844572,202205,12070.0,1.51,7993.38,9.41,6.42,57.0,3.65940,NaN,1.019546,1.019546,1.019546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130229,NONE,EX,RD,K,SI1,EX,EX,15409661,202207,4300.0,1.08,3981.48,6.55,6.56,62.1,4.07376,1.000000,1.000000,1.000000,1.000000
130230,NONE,EX,RD,I,SI1,EX,EX,15409696,202207,6040.0,1.01,5980.20,6.46,6.49,60.8,3.94592,1.000000,1.000000,1.000000,1.000000
130231,MED,EX,OV,H,VVS2,None,EX,15410303,202207,6620.0,1.01,6554.46,7.76,5.40,65.0,3.51000,NaN,1.000000,1.000000,1.000000
130232,NONE,VG,EM,D,VS1,None,EX,15410668,202207,14750.0,1.50,9833.33,7.84,5.43,65.8,3.57294,NaN,1.000000,1.000000,1.000000


In [ ]:
df_with_inf.inflation.fillna(df_with_inf.inflation_2cat_group[df_with_inf.inflation.isna() == True], inplace=True)
df_with_inf

,fluor,symmetry,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y,inflation,inflation_2cat_group,inflation_3cat_group,inflation_4cat_group
0,NONE,VG,EM,K,SI2,NaN,VG,12902293,202205,1390.0,0.91,1527.47,6.30,4.58,71.0,3.25180,1.037988,1.037988,1.037988,1.037988
1,MED,EX,RD,J,SI2,EX,EX,14584509,202205,3700.0,1.01,3663.37,6.37,6.42,62.0,3.98040,1.003894,1.015418,1.013542,1.015676
2,NONE,EX,RD,G,SI1,EX,EX,14773748,202206,7170.0,1.00,7170.00,6.30,6.35,63.7,4.04495,1.013707,1.008800,1.009436,1.010882
3,MED,EX,OV,H,VS2,NaN,EX,15281430,202207,5160.0,1.00,5160.00,7.43,5.64,64.6,3.64344,1.000000,1.000000,1.000000,1.000000
4,NONE,VG,OV,G,SI1,NaN,EX,14844572,202205,12070.0,1.51,7993.38,9.41,6.42,57.0,3.65940,1.019546,1.019546,1.019546,1.019546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130229,NONE,EX,RD,K,SI1,EX,EX,15409661,202207,4300.0,1.08,3981.48,6.55,6.56,62.1,4.07376,1.000000,1.000000,1.000000,1.000000
130230,NONE,EX,RD,I,SI1,EX,EX,15409696,202207,6040.0,1.01,5980.20,6.46,6.49,60.8,3.94592,1.000000,1.000000,1.000000,1.000000
130231,MED,EX,OV,H,VVS2,None,EX,15410303,202207,6620.0,1.01,6554.46,7.76,5.40,65.0,3.51000,1.000000,1.000000,1.000000,1.000000
130232,NONE,VG,EM,D,VS1,None,EX,15410668,202207,14750.0,1.50,9833.33,7.84,5.43,65.8,3.57294,1.000000,1.000000,1.000000,1.000000


In [ ]:
# Приведем цены к 202207 с учетом инфляции
df_with_inf["price_per_carat_202207"] = df_with_inf.price_per_carat * df_with_inf.inflation
df_with_inf

,fluor,symmetry,shape,color,clarity,cut,polish,id,date,price,...,price_per_carat,z,x,depth_perc,y,inflation,inflation_2cat_group,inflation_3cat_group,inflation_4cat_group,price_per_carat_202207
0,NONE,VG,EM,K,SI2,NaN,VG,12902293,202205,1390.0,...,1527.47,6.30,4.58,71.0,3.25180,1.037988,1.037988,1.037988,1.037988,1585.494868
1,MED,EX,RD,J,SI2,EX,EX,14584509,202205,3700.0,...,3663.37,6.37,6.42,62.0,3.98040,1.003894,1.015418,1.013542,1.015676,3677.634025
2,NONE,EX,RD,G,SI1,EX,EX,14773748,202206,7170.0,...,7170.00,6.30,6.35,63.7,4.04495,1.013707,1.008800,1.009436,1.010882,7268.276552
3,MED,EX,OV,H,VS2,NaN,EX,15281430,202207,5160.0,...,5160.00,7.43,5.64,64.6,3.64344,1.000000,1.000000,1.000000,1.000000,5160.000000
4,NONE,VG,OV,G,SI1,NaN,EX,14844572,202205,12070.0,...,7993.38,9.41,6.42,57.0,3.65940,1.019546,1.019546,1.019546,1.019546,8149.618234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130229,NONE,EX,RD,K,SI1,EX,EX,15409661,202207,4300.0,...,3981.48,6.55,6.56,62.1,4.07376,1.000000,1.000000,1.000000,1.000000,3981.480000
130230,NONE,EX,RD,I,SI1,EX,EX,15409696,202207,6040.0,...,5980.20,6.46,6.49,60.8,3.94592,1.000000,1.000000,1.000000,1.000000,5980.200000
130231,MED,EX,OV,H,VVS2,None,EX,15410303,202207,6620.0,...,6554.46,7.76,5.40,65.0,3.51000,1.000000,1.000000,1.000000,1.000000,6554.460000
130232,NONE,VG,EM,D,VS1,None,EX,15410668,202207,14750.0,...,9833.33,7.84,5.43,65.8,3.57294,1.000000,1.000000,1.000000,1.000000,9833.330000


In [ ]:
# удалим столбцы "price" - напрямую зависит от цены за карат, "date" - приведено к 202207, "id" - уникальные значения, inflation
# удалим price_per_carat - тк есть price_per_carat_202207
df = df_with_inf.drop(columns=["price", "price_per_carat", "date", "id", "inflation", "inflation_2cat_group", "inflation_3cat_group", "inflation_4cat_group"])
df

,fluor,symmetry,shape,color,clarity,cut,polish,carat,z,x,depth_perc,y,price_per_carat_202207
0,NONE,VG,EM,K,SI2,NaN,VG,0.91,6.30,4.58,71.0,3.25180,1585.494868
1,MED,EX,RD,J,SI2,EX,EX,1.01,6.37,6.42,62.0,3.98040,3677.634025
2,NONE,EX,RD,G,SI1,EX,EX,1.00,6.30,6.35,63.7,4.04495,7268.276552
3,MED,EX,OV,H,VS2,NaN,EX,1.00,7.43,5.64,64.6,3.64344,5160.000000
4,NONE,VG,OV,G,SI1,NaN,EX,1.51,9.41,6.42,57.0,3.65940,8149.618234
...,...,...,...,...,...,...,...,...,...,...,...,...,...
130229,NONE,EX,RD,K,SI1,EX,EX,1.08,6.55,6.56,62.1,4.07376,3981.480000
130230,NONE,EX,RD,I,SI1,EX,EX,1.01,6.46,6.49,60.8,3.94592,5980.200000
130231,MED,EX,OV,H,VVS2,None,EX,1.01,7.76,5.40,65.0,3.51000,6554.460000
130232,NONE,VG,EM,D,VS1,None,EX,1.50,7.84,5.43,65.8,3.57294,9833.330000


## Задание 2: Модель

Определите функцию потерь (MSE или MAE) и аргументируйте выбор.
Попробуйте сделать baseline.
Используйте LabelEncoder для категориальных фичей и постройте линейную модель.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error # Выберем MAE, чтобы смотреть разницу в цене в той же размерности
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
# Разбиваем датасет
X = df.drop(columns=["price_per_carat_202207"])
y = df["price_per_carat_202207"]
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)


In [ ]:
# Ошибки моделей
diamonds_scores = {}

In [ ]:
# baseline - средняя цена
mean_price_per_carat = y_train.mean()
diamonds_scores['baseline_mae'] = mean_absolute_error(y_valid, [mean_price_per_carat]*len(y_valid))
diamonds_scores['baseline_mae']

2128.0725080274115

In [ ]:
df

,fluor,symmetry,shape,color,clarity,cut,polish,carat,z,x,depth_perc,y,price_per_carat_202207
0,NONE,VG,EM,K,SI2,NaN,VG,0.91,6.30,4.58,71.0,3.25180,1585.494868
1,MED,EX,RD,J,SI2,EX,EX,1.01,6.37,6.42,62.0,3.98040,3677.634025
2,NONE,EX,RD,G,SI1,EX,EX,1.00,6.30,6.35,63.7,4.04495,7268.276552
3,MED,EX,OV,H,VS2,NaN,EX,1.00,7.43,5.64,64.6,3.64344,5160.000000
4,NONE,VG,OV,G,SI1,NaN,EX,1.51,9.41,6.42,57.0,3.65940,8149.618234
...,...,...,...,...,...,...,...,...,...,...,...,...,...
130229,NONE,EX,RD,K,SI1,EX,EX,1.08,6.55,6.56,62.1,4.07376,3981.480000
130230,NONE,EX,RD,I,SI1,EX,EX,1.01,6.46,6.49,60.8,3.94592,5980.200000
130231,MED,EX,OV,H,VVS2,None,EX,1.01,7.76,5.40,65.0,3.51000,6554.460000
130232,NONE,VG,EM,D,VS1,None,EX,1.50,7.84,5.43,65.8,3.57294,9833.330000


In [ ]:
# Используйте LabelEncoder для категориальных фичей
df_le = df.copy()
for param in ['fluor', 'symmetry', 'shape', 'color', 'clarity', 'cut', 'polish']:
    df_le[param] = LabelEncoder().fit_transform(y=df_le[param])
    
df_le

,fluor,symmetry,shape,color,clarity,cut,polish,carat,z,x,depth_perc,y,price_per_carat_202207
0,2,3,1,7,4,4,2,0.91,6.30,4.58,71.0,3.25180,1585.494868
1,1,0,9,6,4,0,0,1.01,6.37,6.42,62.0,3.98040,3677.634025
2,2,0,9,3,3,0,0,1.00,6.30,6.35,63.7,4.04495,7268.276552
3,1,0,5,4,6,4,0,1.00,7.43,5.64,64.6,3.64344,5160.000000
4,2,3,5,3,3,4,0,1.51,9.41,6.42,57.0,3.65940,8149.618234
...,...,...,...,...,...,...,...,...,...,...,...,...,...
130229,2,0,9,7,3,0,0,1.08,6.55,6.56,62.1,4.07376,3981.480000
130230,2,0,9,5,3,0,0,1.01,6.46,6.49,60.8,3.94592,5980.200000
130231,1,0,5,4,8,3,0,1.01,7.76,5.40,65.0,3.51000,6554.460000
130232,2,3,1,0,5,3,0,1.50,7.84,5.43,65.8,3.57294,9833.330000


In [ ]:
# Линейная регрессия
X = df_le.drop(columns=["price_per_carat_202207"])
y = df_le["price_per_carat_202207"]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)


lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred = lin_reg.predict(X_valid)
diamonds_scores['lin_reg_mae'] = mean_absolute_error(y_valid, y_pred)
diamonds_scores['lin_reg_mae'] 

1179.2625171853526

Теперь попробуйте OHE или TargetEncoder (сравните их).

Нормализуйте данные.

Поработайте с пропусками (обратите внимание на то, что у fluor возможен пропуск значения, а возможно отсутствие флуоресценции)

Покажите, насколько получилось улучшить результат.

In [ ]:
# !pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 758 kB/s 


In [ ]:
from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler

from typing import Tuple, List


def OHE(df: pd.DataFrame, columns: List[str]) -> Tuple[pd.DataFrame, List[str]]:
    index = df.index
    one = OneHotEncoder(sparse=False, categories='auto')
    ohe = one.fit_transform(df[columns])
    col_names = one.get_feature_names(input_features = columns)
    df = df.drop(columns, axis=1)
    df = df.reset_index(drop=True)
    df = pd.concat([df, pd.DataFrame(ohe, columns=col_names)], axis = 1)
    df = df.set_index(index)
    return (df, col_names)

scaler = StandardScaler() # Помните, что на тесте делаем только transform
  

In [ ]:
# Разбиваем датасет
X = df.drop(columns=["price_per_carat_202207"])
y = df["price_per_carat_202207"]


In [ ]:
# OneHotEncoder
cat_features = X.columns[X.dtypes == object]
X, cat_col = OHE(X, cat_features)
X

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,carat,z,x,depth_perc,y,fluor_FNT,fluor_MED,fluor_NONE,fluor_STG,fluor_nan,...,clarity_VVS1,clarity_VVS2,cut_EX,cut_G,cut_VG,cut_None,cut_nan,polish_EX,polish_G,polish_VG
0,0.91,6.30,4.58,71.0,3.25180,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,1.01,6.37,6.42,62.0,3.98040,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.00,6.30,6.35,63.7,4.04495,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.00,7.43,5.64,64.6,3.64344,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,1.51,9.41,6.42,57.0,3.65940,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130229,1.08,6.55,6.56,62.1,4.07376,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
130230,1.01,6.46,6.49,60.8,3.94592,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
130231,1.01,7.76,5.40,65.0,3.51000,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
130232,1.50,7.84,5.43,65.8,3.57294,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [ ]:
# Линейная регрессия после ohe

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred = lin_reg.predict(X_valid)
diamonds_scores['lin_reg_mae_ohe'] = mean_absolute_error(y_valid, y_pred)
diamonds_scores['lin_reg_mae_ohe']

1179.2625171853526

In [ ]:
# Нормализация
X = pd.DataFrame(scaler.fit_transform(X),columns = X.columns)

In [ ]:
# Линейная регрессия после ohe  и нормализации

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred = lin_reg.predict(X_valid)
diamonds_scores['lin_reg_mae_ohe_norm'] = mean_absolute_error(y_valid, y_pred)
diamonds_scores['lin_reg_mae_ohe_norm'] 


1179.2625171853529

Сравните с KNN

In [ ]:
# knn после ohe и нормализации
knn = KNeighborsRegressor(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_valid)
diamonds_scores['knn_mae_ohe_norm'] = mean_absolute_error(y_valid, y_pred)
diamonds_scores['knn_mae_ohe_norm']

648.7257414202742

In [ ]:
df

,fluor,symmetry,shape,color,clarity,cut,polish,carat,z,x,depth_perc,y,price_per_carat_202207
0,NONE,VG,EM,K,SI2,NaN,VG,0.91,6.30,4.58,71.0,3.25180,1585.494868
1,MED,EX,RD,J,SI2,EX,EX,1.01,6.37,6.42,62.0,3.98040,3677.634025
2,NONE,EX,RD,G,SI1,EX,EX,1.00,6.30,6.35,63.7,4.04495,7268.276552
3,MED,EX,OV,H,VS2,NaN,EX,1.00,7.43,5.64,64.6,3.64344,5160.000000
4,NONE,VG,OV,G,SI1,NaN,EX,1.51,9.41,6.42,57.0,3.65940,8149.618234
...,...,...,...,...,...,...,...,...,...,...,...,...,...
130229,NONE,EX,RD,K,SI1,EX,EX,1.08,6.55,6.56,62.1,4.07376,3981.480000
130230,NONE,EX,RD,I,SI1,EX,EX,1.01,6.46,6.49,60.8,3.94592,5980.200000
130231,MED,EX,OV,H,VVS2,None,EX,1.01,7.76,5.40,65.0,3.51000,6554.460000
130232,NONE,VG,EM,D,VS1,None,EX,1.50,7.84,5.43,65.8,3.57294,9833.330000


In [ ]:
# TargetEncoder
df_te = df.copy()
for param in ['fluor', 'symmetry', 'shape', 'color', 'clarity', 'cut', 'polish']:
    df_te[param] = TargetEncoder().fit_transform(df_te[param], df_te.price_per_carat_202207)
    
df_te


,fluor,symmetry,shape,color,clarity,cut,polish,carat,z,x,depth_perc,y,price_per_carat_202207
0,6456.643823,5432.509389,5363.011728,3955.939104,4797.794132,5056.341247,5094.065769,0.91,6.30,4.58,71.0,3.25180,1585.494868
1,6165.888044,6909.069322,6986.518196,4757.259637,4797.794132,7262.539314,6478.055911,1.01,6.37,6.42,62.0,3.98040,3677.634025
2,6456.643823,6909.069322,6986.518196,6767.611475,5810.077469,7262.539314,6478.055911,1.00,6.30,6.35,63.7,4.04495,7268.276552
3,6165.888044,6909.069322,5190.513284,6367.391513,6840.281210,5056.341247,6478.055911,1.00,7.43,5.64,64.6,3.64344,5160.000000
4,6456.643823,5432.509389,5190.513284,6767.611475,5810.077469,5056.341247,6478.055911,1.51,9.41,6.42,57.0,3.65940,8149.618234
...,...,...,...,...,...,...,...,...,...,...,...,...,...
130229,6456.643823,6909.069322,6986.518196,3955.939104,5810.077469,7262.539314,6478.055911,1.08,6.55,6.56,62.1,4.07376,3981.480000
130230,6456.643823,6909.069322,6986.518196,5637.816143,5810.077469,7262.539314,6478.055911,1.01,6.46,6.49,60.8,3.94592,5980.200000
130231,6165.888044,6909.069322,5190.513284,6367.391513,7471.128102,5056.341247,6478.055911,1.01,7.76,5.40,65.0,3.51000,6554.460000
130232,6456.643823,5432.509389,5363.011728,7803.522295,7248.247533,5056.341247,6478.055911,1.50,7.84,5.43,65.8,3.57294,9833.330000


In [ ]:
# Разбиваем датасет
X = df_te.drop(columns=["price_per_carat_202207"])
y = df_te["price_per_carat_202207"]


In [ ]:
# Линейная регрессия после te

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred = lin_reg.predict(X_valid)
diamonds_scores['lin_reg_mae_te'] = mean_absolute_error(y_valid, y_pred)
diamonds_scores['lin_reg_mae_te']

888.7872034504244

In [ ]:
# Нормализация
X = pd.DataFrame(scaler.fit_transform(X),columns = X.columns)
X

,fluor,symmetry,shape,color,clarity,cut,polish,carat,z,x,depth_perc,y
0,0.735463,-1.170296,-1.056326,-2.044903,-1.327930,-1.27145,-2.638540,-0.840507,-0.573749,-2.271901,2.234805,-1.799665
1,-0.693029,0.805830,0.760485,-1.347915,-1.327930,0.97152,0.372236,-0.508666,-0.494595,0.209451,-0.500705,0.012026
2,0.735463,0.805830,0.760485,0.400686,-0.437204,0.97152,0.372236,-0.541850,-0.573749,0.115052,0.016002,0.172532
3,-0.693029,0.805830,-1.249363,0.052575,0.469290,-1.27145,0.372236,-0.541850,0.704025,-0.842426,0.289553,-0.825837
4,0.735463,-1.170296,-1.249363,0.400686,-0.437204,-1.27145,0.372236,1.150540,2.942957,0.209451,-2.020433,-0.786152
...,...,...,...,...,...,...,...,...,...,...,...,...
130229,0.735463,0.805830,0.760485,-2.044903,-0.437204,0.97152,0.372236,-0.276377,-0.291056,0.398250,-0.470311,0.244169
130230,0.735463,0.805830,0.760485,-0.582009,-0.437204,0.97152,0.372236,-0.508666,-0.392826,0.303850,-0.865440,-0.073710
130231,-0.693029,0.805830,-1.249363,0.052575,1.024384,-1.27145,0.372236,-0.508666,1.077180,-1.166081,0.411131,-1.157641
130232,0.735463,-1.170296,-1.056326,1.301719,0.828267,-1.27145,0.372236,1.117356,1.167642,-1.125624,0.654288,-1.001138


In [ ]:
# Линейная регрессия после te  и нормализации
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred = lin_reg.predict(X_valid)
diamonds_scores['lin_reg_mae_te_norm'] = mean_absolute_error(y_valid, y_pred)
diamonds_scores['lin_reg_mae_te_norm']

888.7872034504245

In [ ]:
# knn после te и нормализации
knn = KNeighborsRegressor(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_valid)
diamonds_scores['knn_mae_te_norm'] = mean_absolute_error(y_valid, y_pred)
diamonds_scores['knn_mae_te_norm']

614.7504883067899

In [ ]:
diamonds_scores

{'baseline_mae': 2128.0725080274115,
 'lin_reg_mae': 1179.2625171853526,
 'lin_reg_mae_ohe': 1179.2625171853526,
 'lin_reg_mae_ohe_norm': 1179.2625171853529,
 'knn_mae_ohe_norm': 648.7257414202742,
 'lin_reg_mae_te': 888.7872034504244,
 'lin_reg_mae_te_norm': 888.7872034504245,
 'knn_mae_te_norm': 614.7504883067899}

**Вывод**: knn модель с target engoder и one hot encoder дает наименьшую ошибку.

# Классификация

Загрузите датасет bodyPerformance

Описание:

https://www.kaggle.com/datasets/kukuroo3/body-performance-data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = 'drive/MyDrive/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_body = pd.read_csv(path + 'bodyPerformance.csv') #укажите свой путь
df_body

,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class
0,27.0,M,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,C
1,25.0,M,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,A
2,31.0,M,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,C
3,32.0,M,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,B
4,28.0,M,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,B
...,...,...,...,...,...,...,...,...,...,...,...,...
13388,25.0,M,172.1,71.80,16.2,74.0,141.0,35.8,17.4,47.0,198.0,C
13389,21.0,M,179.7,63.90,12.1,74.0,128.0,33.0,1.1,48.0,167.0,D
13390,39.0,M,177.2,80.50,20.1,78.0,132.0,63.5,16.4,45.0,229.0,A
13391,64.0,F,146.1,57.70,40.4,68.0,121.0,19.3,9.2,0.0,75.0,D


## Задание 1: Определение гендера

Постройте модель, которая будет определять гендер.

Для этого определите две метрики качества, на которые вы будете смотреть. Аргументируйте свой выбор.

После чего преобразуйте категориальные переменные и постройте модель бинарной классификации на основе линейного классификатора и сравните с KNN.

P.S.: не забудте про нормализацию.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

scaler = StandardScaler()

In [ ]:
df_body.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13393 entries, 0 to 13392
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      13393 non-null  float64
 1   gender                   13393 non-null  object 
 2   height_cm                13393 non-null  float64
 3   weight_kg                13393 non-null  float64
 4   body fat_%               13393 non-null  float64
 5   diastolic                13393 non-null  float64
 6   systolic                 13393 non-null  float64
 7   gripForce                13393 non-null  float64
 8   sit and bend forward_cm  13393 non-null  float64
 9   sit-ups counts           13393 non-null  float64
 10  broad jump_cm            13393 non-null  float64
 11  class                    13393 non-null  object 
dtypes: float64(10), object(2)
memory usage: 1.2+ MB


In [ ]:
# Разделим выборки на параметры и целевую переменную
X = df_body.drop(columns=["gender"])
y = df_body["gender"]

# Преобразуем целевые категории к цифрам
label_to_gender = {
    'F': 0,
    'M': 1
}
y = np.array([label_to_gender[i] for i in y])

y

array([1, 1, 1, ..., 1, 0, 1])

In [ ]:
# преобразуем категориальные переменные с помощью OHE
cat_features = X.columns[X.dtypes == object]
X, cat_col = OHE(X, cat_features)
X

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,age,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class_A,class_B,class_C,class_D
0,27.0,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,0.0,0.0,1.0,0.0
1,25.0,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,1.0,0.0,0.0,0.0
2,31.0,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,0.0,0.0,1.0,0.0
3,32.0,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,0.0,1.0,0.0,0.0
4,28.0,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13388,25.0,172.1,71.80,16.2,74.0,141.0,35.8,17.4,47.0,198.0,0.0,0.0,1.0,0.0
13389,21.0,179.7,63.90,12.1,74.0,128.0,33.0,1.1,48.0,167.0,0.0,0.0,0.0,1.0
13390,39.0,177.2,80.50,20.1,78.0,132.0,63.5,16.4,45.0,229.0,1.0,0.0,0.0,0.0
13391,64.0,146.1,57.70,40.4,68.0,121.0,19.3,9.2,0.0,75.0,0.0,0.0,0.0,1.0


In [ ]:
# Разбиваем на трейн и валид выборки
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)

In [ ]:
# Оценка моделей
body_scores = {
    'roc_auc': {},
    'f1': {},
    'accuracy': {},
    'precision': {}
}

def get_scores(y_valid, y_pred, model):
    body_scores['roc_auc'][model] = metrics.roc_auc_score(y_valid, y_pred)
    body_scores['f1'][model] = metrics.f1_score(y_valid, y_pred)
    body_scores['accuracy'][model] = metrics.accuracy_score(y_valid, y_pred)
    body_scores['precision'][model] = metrics.precision_score(y_valid, y_pred)

In [ ]:
# knn после ohe и до нормализации
knn = KNeighborsRegressor(n_neighbors=59)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_valid)

y_pred = y_pred.round()

get_scores(y_valid, y_pred, 'knn_ohe')

body_scores

{'roc_auc': {'knn_ohe': 0.9847220947587559},
 'f1': {'knn_ohe': 0.9882835383714118},
 'accuracy': {'knn_ohe': 0.9850690556177678},
 'precision': {'knn_ohe': 0.9906048150322959}}

In [ ]:
# log_reg после ohe и до нормализации
log_reg = LogisticRegression(random_state=5, n_jobs=-1)
log_reg = log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_valid)

get_scores(y_valid, y_pred, 'log_reg_ohe')
body_scores

{'roc_auc': {'knn_ohe': 0.9847220947587559, 'log_reg_ohe': 0.9849258685897282},
 'f1': {'knn_ohe': 0.9882835383714118, 'log_reg_ohe': 0.9891907683318727},
 'accuracy': {'knn_ohe': 0.9850690556177678,
  'log_reg_ohe': 0.9861888764464353},
 'precision': {'knn_ohe': 0.9906048150322959,
  'log_reg_ohe': 0.9889018691588785}}

In [ ]:
# Нормализация
scaler.fit(X_train, y_train)
X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
X_valid = pd.DataFrame(scaler.transform(X_valid), index=X_valid.index, columns=X_valid.columns)

X_train

,age,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class_A,class_B,class_C,class_D
9269,-0.650446,-0.719235,-0.553326,1.045595,-0.348531,-1.644221,-1.061516,0.837737,0.157928,-0.452194,1.727107,-0.574261,-0.580152,-0.575985
11306,0.819564,0.397030,0.434846,0.216805,1.794865,1.010604,0.418681,-0.010609,0.227818,0.549860,-0.579003,1.741369,-0.580152,-0.575985
12740,-1.164950,-0.849862,-0.829680,1.832945,-0.907677,-1.235787,-1.117726,0.777995,-0.680754,-1.704763,-0.579003,-0.574261,1.723685,-0.575985
2557,1.995573,-2.227380,-0.235101,2.813679,-0.255340,1.078676,-1.876561,-0.285426,-2.637679,-2.681766,-0.579003,-0.574261,-0.580152,1.736156
10718,-0.797447,1.240166,2.511684,1.128474,-0.068957,-0.010483,1.383620,0.706303,0.367599,0.123987,-0.579003,-0.574261,-0.580152,1.736156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866,-1.017949,-0.184853,-0.653819,-1.896608,-1.000869,-0.623135,0.287525,0.013288,-0.540974,0.549860,-0.579003,-0.574261,-0.580152,1.736156
10982,-0.576945,-0.647984,-0.544952,1.708626,-1.280442,-1.167714,-0.546257,-0.524396,-0.540974,-1.103530,-0.579003,-0.574261,1.723685,-0.575985
7696,1.040066,-0.042351,0.016129,0.838397,1.888056,1.555183,-0.358891,-0.381014,0.437489,-0.076424,-0.579003,1.741369,-0.580152,-0.575985
74,-1.091449,0.432655,0.267359,0.147739,-0.814486,-0.827352,-0.021630,-0.643882,0.717049,0.574911,-0.579003,-0.574261,-0.580152,1.736156


In [ ]:
# knn после ohe и нормализации
knn = KNeighborsRegressor(n_neighbors=59)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_valid)

y_pred = y_pred.round()

get_scores(y_valid, y_pred, 'knn_ohe_norm')
body_scores

{'roc_auc': {'knn_ohe': 0.9847220947587559,
  'log_reg_ohe': 0.9849258685897282,
  'knn_ohe_norm': 0.9860947756616159},
 'f1': {'knn_ohe': 0.9882835383714118,
  'log_reg_ohe': 0.9891907683318727,
  'knn_ohe_norm': 0.9903705865188211},
 'accuracy': {'knn_ohe': 0.9850690556177678,
  'log_reg_ohe': 0.9861888764464353,
  'knn_ohe_norm': 0.9876819708846585},
 'precision': {'knn_ohe': 0.9906048150322959,
  'log_reg_ohe': 0.9889018691588785,
  'knn_ohe_norm': 0.9889277389277389}}

In [ ]:
# log_reg после ohe и нормализации
log_reg = LogisticRegression(random_state=5, n_jobs=-1)
log_reg = log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_valid)

get_scores(y_valid, y_pred, 'log_reg_ohe_norm')
body_scores

{'roc_auc': {'knn_ohe': 0.9847220947587559,
  'log_reg_ohe': 0.9849258685897282,
  'knn_ohe_norm': 0.9860947756616159,
  'log_reg_ohe_norm': 0.9892618738256589},
 'f1': {'knn_ohe': 0.9882835383714118,
  'log_reg_ohe': 0.9891907683318727,
  'knn_ohe_norm': 0.9903705865188211,
  'log_reg_ohe_norm': 0.9924065420560747},
 'accuracy': {'knn_ohe': 0.9850690556177678,
  'log_reg_ohe': 0.9861888764464353,
  'knn_ohe_norm': 0.9876819708846585,
  'log_reg_ohe_norm': 0.990294886151549},
 'precision': {'knn_ohe': 0.9906048150322959,
  'log_reg_ohe': 0.9889018691588785,
  'knn_ohe_norm': 0.9889277389277389,
  'log_reg_ohe_norm': 0.9918272037361354}}

In [ ]:
df_body_scores = pd.DataFrame(data=body_scores)
df_body_scores

,roc_auc,f1,accuracy,precision
knn_ohe,0.984722,0.988284,0.985069,0.990605
log_reg_ohe,0.984926,0.989191,0.986189,0.988902
knn_ohe_norm,0.986095,0.990371,0.987682,0.988928
log_reg_ohe_norm,0.989262,0.992407,0.990295,0.991827


In [ ]:
df_body

,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class
0,27.0,M,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,C
1,25.0,M,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,A
2,31.0,M,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,C
3,32.0,M,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,B
4,28.0,M,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,B
...,...,...,...,...,...,...,...,...,...,...,...,...
13388,25.0,M,172.1,71.80,16.2,74.0,141.0,35.8,17.4,47.0,198.0,C
13389,21.0,M,179.7,63.90,12.1,74.0,128.0,33.0,1.1,48.0,167.0,D
13390,39.0,M,177.2,80.50,20.1,78.0,132.0,63.5,16.4,45.0,229.0,A
13391,64.0,F,146.1,57.70,40.4,68.0,121.0,19.3,9.2,0.0,75.0,D


## Задание 2: Определение класса

Теперь опробуем построить модель, которая будет предсказывать class физической формы.

Для этого определите метрики качества для задачи мультиклассификации (аргументируйте выбор).

Постройте модель мультиклассовой классификации на основе линейного классификатора и сравните с KNN.

In [ ]:
X = df_body.drop(columns=["class"]) # укажите новый таргет
y = df_body["class"]


In [ ]:
# закодируем значения gender
enc = LabelEncoder()
enc.fit(X['gender'])
X['gender'] = enc.transform(X['gender'])
X

,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm
0,27.0,1,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0
1,25.0,1,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0
2,31.0,1,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0
3,32.0,1,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0
4,28.0,1,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0
...,...,...,...,...,...,...,...,...,...,...,...
13388,25.0,1,172.1,71.80,16.2,74.0,141.0,35.8,17.4,47.0,198.0
13389,21.0,1,179.7,63.90,12.1,74.0,128.0,33.0,1.1,48.0,167.0
13390,39.0,1,177.2,80.50,20.1,78.0,132.0,63.5,16.4,45.0,229.0
13391,64.0,0,146.1,57.70,40.4,68.0,121.0,19.3,9.2,0.0,75.0


In [ ]:
# закодируем значения y
enc = LabelEncoder()
enc.fit(y)
y = enc.transform(y)
y

array([2, 0, 2, ..., 0, 3, 2])

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)

In [ ]:
# Оценка моделей
body_class_scores = {
    'f1_micro': {},
    'accuracy': {},
    'precision': {}
}

def get_body_class_scores(y_valid, y_pred, model):
    body_class_scores['f1_micro'][model] = metrics.f1_score(y_valid, y_pred, average='micro')
    body_class_scores['accuracy'][model] = metrics.accuracy_score(y_valid, y_pred)
    body_class_scores['precision'][model] = metrics.precision_score(y_valid, y_pred, average='weighted')

In [ ]:
# log_reg
log_reg = LogisticRegression(random_state=5, n_jobs=-1)
log_reg = log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_valid)

get_body_class_scores(y_valid, y_pred, 'log_reg')
body_class_scores

{'f1_micro': {'log_reg': 0.5968645016797313},
 'accuracy': {'log_reg': 0.5968645016797313},
 'precision': {'log_reg': 0.5956596705387189}}

In [ ]:
# knn
knn = KNeighborsRegressor(n_neighbors=59)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_valid)
y_pred = y_pred.round()

get_body_class_scores(y_valid, y_pred, 'knn')
body_class_scores

{'f1_micro': {'log_reg': 0.5968645016797313, 'knn': 0.49720044792833146},
 'accuracy': {'log_reg': 0.5968645016797313, 'knn': 0.49720044792833146},
 'precision': {'log_reg': 0.5956596705387189, 'knn': 0.649248214718473}}

In [ ]:
# Нормализация
scaler.fit(X_train, y_train)
X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
X_valid = pd.DataFrame(scaler.transform(X_valid), index=X_valid.index, columns=X_valid.columns)

X_train

,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm
9269,-0.650446,-1.306493,-0.719235,-0.553326,1.045595,-0.348531,-1.644221,-1.061516,0.837737,0.157928,-0.452194
11306,0.819564,0.765408,0.397030,0.434846,0.216805,1.794865,1.010604,0.418681,-0.010609,0.227818,0.549860
12740,-1.164950,-1.306493,-0.849862,-0.829680,1.832945,-0.907677,-1.235787,-1.117726,0.777995,-0.680754,-1.704763
2557,1.995573,-1.306493,-2.227380,-0.235101,2.813679,-0.255340,1.078676,-1.876561,-0.285426,-2.637679,-2.681766
10718,-0.797447,0.765408,1.240166,2.511684,1.128474,-0.068957,-0.010483,1.383620,0.706303,0.367599,0.123987
...,...,...,...,...,...,...,...,...,...,...,...
866,-1.017949,0.765408,-0.184853,-0.653819,-1.896608,-1.000869,-0.623135,0.287525,0.013288,-0.540974,0.549860
10982,-0.576945,-1.306493,-0.647984,-0.544952,1.708626,-1.280442,-1.167714,-0.546257,-0.524396,-0.540974,-1.103530
7696,1.040066,0.765408,-0.042351,0.016129,0.838397,1.888056,1.555183,-0.358891,-0.381014,0.437489,-0.076424
74,-1.091449,0.765408,0.432655,0.267359,0.147739,-0.814486,-0.827352,-0.021630,-0.643882,0.717049,0.574911


In [ ]:
# log_reg norm
log_reg = LogisticRegression(random_state=5, n_jobs=-1)
log_reg = log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_valid)

get_body_class_scores(y_valid, y_pred, 'log_reg_norm')
body_class_scores

{'f1_micro': {'log_reg': 0.5968645016797313,
  'knn': 0.49720044792833146,
  'log_reg_norm': 0.6151549085479656},
 'accuracy': {'log_reg': 0.5968645016797313,
  'knn': 0.49720044792833146,
  'log_reg_norm': 0.6151549085479656},
 'precision': {'log_reg': 0.5956596705387189,
  'knn': 0.649248214718473,
  'log_reg_norm': 0.6160805683291821}}

In [ ]:
# knn
knn = KNeighborsRegressor(n_neighbors=59)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_valid)
y_pred = y_pred.round()

get_body_class_scores(y_valid, y_pred, 'knn_norm')
body_class_scores

{'f1_micro': {'log_reg': 0.5968645016797313,
  'knn': 0.49720044792833146,
  'log_reg_norm': 0.6151549085479656,
  'knn_norm': 0.5617767823814857},
 'accuracy': {'log_reg': 0.5968645016797313,
  'knn': 0.49720044792833146,
  'log_reg_norm': 0.6151549085479656,
  'knn_norm': 0.5617767823814857},
 'precision': {'log_reg': 0.5956596705387189,
  'knn': 0.649248214718473,
  'log_reg_norm': 0.6160805683291821,
  'knn_norm': 0.6756782235241456}}

In [ ]:
df_body_class_scores = pd.DataFrame(data=body_class_scores)
df_body_class_scores

,f1_micro,accuracy,precision
log_reg,0.596865,0.596865,0.595660
knn,0.497200,0.497200,0.649248
log_reg_norm,0.615155,0.615155,0.616081
knn_norm,0.561777,0.561777,0.675678
